<a href="https://colab.research.google.com/github/arun7pulse/askindex/blob/master/NIfty50Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install required Packages**

In [ ]:
%load_ext google.colab.data_table


# **Import required library**

In [ ]:
""" Author - ArunSK. #nsenifty.py """
import datetime
import logging
import sys, os, time, json
import requests
import sqlite3 as lite
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8'}
#import pymongo
#from asklib import *
#from pymongo.errors import BulkWriteError, ConnectionFailure

import numpy as np  # Low-level numerical Python library.
import pandas as pd  # Higher-level numerical Python library.
import altair as alt

np.warnings.filterwarnings('ignore')
pd.set_option('expand_frame_repr', False)
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', 18)
pd.set_option('display.max_rows', 40)
pd.set_option('display.width', 15)
pd.options.display.float_format = '{:.2f}'.format

# **Required Functions**

In [ ]:
def split_date_range(start, end, pdays=92):
    firstDate = datetime.datetime.strptime(start, "%d-%m-%Y")
    lastDate = datetime.datetime.strptime(end, "%d-%m-%Y")
    startdate = firstDate
    startdatelist = []
    enddatelist = []
    while startdate <= lastDate:
        enddate = startdate + datetime.timedelta(days=pdays - 1)
        startdatelist.append(startdate.strftime("%d-%m-%Y"))
        if enddate > lastDate: enddatelist.append(lastDate.strftime("%d-%m-%Y"))
        enddatelist.append(enddate.strftime("%d-%m-%Y"))
        startdate = enddate + datetime.timedelta(days=1)
    for sdate, edate in zip(startdatelist, enddatelist):
        yield sdate ,edate

In [ ]:
def csvIndexData(symbol="NIFTY 50", file=""):
    file='{}_01-01-2000_31-12-2019.csv'.format(symbol) if file == "" else file
    if symbol == "NIFTY 50"  :
        file = "https://raw.githubusercontent.com/arun7pulse/askindex/master/NIFTY%2050_01-01-2000_31-12-2019.csv"       
        file = "https://raw.githubusercontent.com/arun7pulse/askindex/master/NIFTY%2050.csv"       
    if symbol == "NIFTY BANK":
        file = "https://raw.githubusercontent.com/arun7pulse/askindex/master/NIFTY%20BANK_01-01-2000_31-12-2019.csv"
        file = "https://raw.githubusercontent.com/arun7pulse/askindex/master/NIFTY%20BANK.csv"
    df = pd.read_csv(file, parse_dates=True, index_col='date', dayfirst=True, error_bad_lines=False).replace("-", method='bfill')
    df =df.astype({"symbol": "category", "open": "float64", "high": "float64", "low": "float64", "close": "float64", "volume": "float64", "value": "float64"} )
    return df

def getIndexData(symbol="NIFTY 50", start=(datetime.datetime.today().date()-datetime.timedelta(days=99)).strftime('%d-%m-%Y'), end=(datetime.datetime.today().date()).strftime('%d-%m-%Y')):
    if start != end:
        url1 = "https://www1.nseindia.com/products/dynaContent/equities/indices/historicalindices.jsp?indexType="
        url2 = symbol.replace(" " , "%20").replace("&", "%26").upper()
        url3 = "&fromDate=" + start + "&toDate=" + end
        url = url1+url2+url3
        print(url)
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                dat = pd.read_html(response.text, header=2)
                df = pd.DataFrame.from_records(dat[0], index='Date')
                df.index.names = ['date']
                df = df[:-1]
                df = df.dropna(axis='rows')
                df.rename(columns={'Open': 'open','High' : 'high', 'Low' : 'low', 'Close' : 'close', 'Shares Traded' : 'volume', 'Turnover ( Cr)' : 'value'}, inplace=True)
                df['symbol'] = symbol
                df.index = pd.to_datetime(df.index)
                df.sort_index(inplace=True)
                return df
        except : 
            return None
    print("start date and end date is same")
    return None

def getIndexFullData(symbol='NIFTY 50', start='01-01-2015', end=(datetime.datetime.today().date()).strftime('%d-%m-%Y')):
    idx = pd.DataFrame()
    idx = idx.append(csvIndexData(symbol=symbol))
    start = (idx.index.max()+datetime.timedelta(days=1)).strftime('%d-%m-%Y')
    for sdate, edate in split_date_range(start, end):
        print(symbol, sdate, edate)
        import time
        time.sleep(0.01)
        df = getIndexData(symbol=symbol, start=sdate, end=edate)
        idx = idx.append(df, verify_integrity=True)
    try:
        idx.to_csv("{}.csv".format(symbol, start, end))
    except :
        pass
    try:
        idx = idx.replace("-", method='bfill')
        idx = idx.astype({"symbol": "category", "open": "float64", "high": "float64", "low": "float64", "close": "float64", "volume": "float64", "value": "float64"} )
    except:
        pass
    return idx

# **Index Class Object**

In [ ]:
class Indices(object):
    def __init__(self, symbol="NIFTY 50", db='.askindex.db'):
        self.conn = lite.connect(db)
        self.cur = self.conn.cursor()
        self.symbol = symbol
    def load_data(self):
        self.df = getIndexFullData(symbol=self.symbol)
        self.df.to_sql(self.symbol, self.conn, if_exists='replace')
        self.dff = self.df.drop('symbol', axis=1)
    def df_frequency(self, sample='W-THU'):
        self.dfw = self.df.resample(sample, label='right').agg({'open':'first', 'high':'max','low':'min','close':'last', 'volume':'sum', 'value':'sum'})
        self.dfwm = self.dfw.resample("BM", label='right').agg({'open':'first', 'high':'max','low':'min','close':'last', 'volume':'sum', 'value':'sum'})
        nesample = "W-FRI"
        self.dfwf = self.df.resample(nesample, label='right').agg({'open':'first', 'high':'max','low':'min','close':'last', 'volume':'sum', 'value':'sum'})
        self.dfwfm = self.dfwf.resample("BM", label='right').agg({'open':'first', 'high':'max','low':'min','close':'last', 'volume':'sum', 'value':'sum'})
    def change(self):
        self.dfwp = (self.dfw.pct_change()*100)[2:]
        self.dfwmp = (self.dfwm.pct_change()*100)[2:]
        self.dfw_target = self.dfwp
        self.dfwm_target = self.dfwmp 

# **Create Instance For Index Symbol**

In [ ]:
nf = Indices()
bf = Indices(symbol = "NIFTY BANK")

# **Load Data from History and from NSE Site**

## **NIFTY Insights for Weekly/Monthly for Option Writing**

In [ ]:
nff = csvIndexData()
print(nff.tail(10))
alt.Chart(nff.tail(2000).reset_index()).mark_line().encode(
  x='date:T',
  y='close',
  color='symbol'
).interactive(bind_y=False)


In [ ]:
nf.df = nff['2010-01-01':]
nf.df_frequency()
nf.change()
print(nf.dfw.tail(10))

alt.Chart(nf.dfw.tail(2000).reset_index()).mark_line().encode(
  x='date:T',
  y='close'
).interactive(bind_y=False)

In [ ]:
#@title NIFTY Weekly(Thu) - Upside/Downside Distribution
UpSidePercent = 98  #@param {type: "slider", min: 0, max: 100}
dwp_upvalue=nf.dfwp[nf.dfwp['close'] > 0 ]['close'].quantile(UpSidePercent/100)
print("\nUPSide Distribution", dwp_upvalue)
DownSidePercent = 98  #@param {type: "slider", min: 0, max: 100}
dwp_downvalue = nf.dfwp[nf.dfwp['close'] < 0 ]['close'].quantile((1-DownSidePercent/100))
print("\nDownSide Distribution", dwp_downvalue)
alt.Chart(nf.dfwp).mark_bar().encode(
  x=alt.X('close', bin=True),
  y='count()',
)



In [ ]:
#@title NIFTY Monthly(Thu) - Upside/Downside Distribution
UpSidePercent = 98  #@param {type: "slider", min: 0, max: 100}
print("\nUPSide Distribution", nf.dfwmp[nf.dfwmp['close'] > 0 ]['close'].quantile(UpSidePercent/100))
DownSidePercent = 98  #@param {type: "slider", min: 0, max: 100}
print("\nDownSide Distribution", nf.dfwmp[nf.dfwmp['close'] < 0 ]['close'].quantile((1-DownSidePercent/100)))
print("\n")
alt.Chart(nf.dfwmp).mark_bar().encode(
  x=alt.X('close', bin=True),
  y='count()',
)

## **NIFTY BANK Insights for Weekly/Monthly for Option Writing**

In [ ]:
bff = csvIndexData("NIFTY BANK")
print(bff.tail(10))
alt.Chart(bff.tail(2000).reset_index()).mark_line().encode(
  x='date:T',
  y='close',
  color='symbol'
).interactive(bind_y=False)

In [ ]:
bf.df = bff['2010-01-01':]
bf.df_frequency()
bf.change()
print(bf.dfw.tail(10))

alt.Chart(bf.dfw.tail(2000).reset_index()).mark_line().encode(
  x='date:T',
  y='close'
).interactive(bind_y=False)

In [ ]:
#@title NIFTY BANK Weekly(Thu) - Upside/Downside Distribution
UpSidePercent = 90  #@param {type: "slider", min: 0, max: 100}
dwp_upvalue=bf.dfwp[bf.dfwp['close'] > 0 ]['close'].quantile(UpSidePercent/100)
print("\nUPSide Distribution", dwp_upvalue)
DownSidePercent = 90  #@param {type: "slider", min: 0, max: 100}
dwp_downvalue = bf.dfwp[bf.dfwp['close'] < 0 ]['close'].quantile((1-DownSidePercent/100))
print("\nDownSide Distribution", dwp_downvalue)
alt.Chart(bf.dfwp).mark_bar().encode(
  x=alt.X('close', bin=True),
  y='count()',
)



In [ ]:
#@title NIFTY BANK Monthly(Thu) - Upside/Downside Distribution
UpSidePercent = 90  #@param {type: "slider", min: 0, max: 100}
print("\nUPSide Distribution", bf.dfwmp[bf.dfwmp['close'] > 0 ]['close'].quantile(UpSidePercent/100))
DownSidePercent = 90  #@param {type: "slider", min: 0, max: 100}
print("\nDownSide Distribution", bf.dfwmp[bf.dfwmp['close'] < 0 ]['close'].quantile((1-DownSidePercent/100)))
print("\n")
alt.Chart(bf.dfwmp).mark_bar().encode(
  x=alt.X('close', bin=True),
  y='count()',
)
